In [1]:
import transformers
import torch
import numpy as np
import random
import pandas as pd
from transformers import AutoTokenizer

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
dataRew=pd.read_json("../Dataset/IMDB_reviews.json",lines=True)

In [4]:
dataRew

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [5]:
dataRew.drop(columns=["movie_id","rating","review_date","user_id","review_summary"],inplace=True)

In [6]:
dataRew

,is_spoiler,review_text
0,True,"In its Oscar year, Shawshank Redemption (writt..."
1,True,The Shawshank Redemption is without a doubt on...
2,True,I believe that this film is the best story eve...
3,True,"**Yes, there are SPOILERS here**This film has ..."
4,True,At the heart of this extraordinary movie is a ...
...,...,...
573908,False,"Go is wise, fast and pure entertainment. Assem..."
573909,False,"Well, what shall I say. this one´s fun at any ..."
573910,False,"Go is the best movie I have ever seen, and I'v..."
573911,False,Call this 1999 teenage version of Pulp Fiction...


### Map true e false, in 0 e 1


In [7]:
dataRew['is_spoiler'] = dataRew['is_spoiler'].map({True: 1, False: 0})

In [8]:
dataRew = dataRew.rename(columns={'is_spoiler': 'label'})

### Split Dataset

In [9]:
import datasets
from datasets import Dataset, DatasetDict

In [10]:
train_temp, test_eval = train_test_split(dataRew, train_size=50000, stratify=dataRew['label'])

# Dividi il temp dataset in test ed evaluation
test, evaluation = train_test_split(test_eval, train_size=25000, stratify=test_eval['label'])

# Dividi il train dataset in train ed evaluation
train, train_eval = train_test_split(train_temp, train_size=25000, stratify=train_temp['label'])

In [11]:
test['label'].value_counts()

label
0    18426
1     6574
Name: count, dtype: int64

In [12]:
test

,label,review_text
297524,0,This movie shows how teenagers confront friend...
256344,0,"The movie was bad, really and honestly now. Th..."
302009,1,A very interesting take on an unconventional d...
22663,0,Forrest Gump was released in a year that inclu...
293916,0,This movie is a gem.It has amazing state-of-th...
...,...,...
488165,0,This movie just blows me away every time. Susa...
328314,0,Because it may well be the worst movie ever ma...
29848,1,I didn't get it. A girl's parents turn into pi...
392835,1,This film came very close to receiving a posit...


In [13]:
train_eval['label'].value_counts()

label
0    18425
1     6575
Name: count, dtype: int64

In [14]:
train_eval

,label,review_text
234302,1,"In this movie, a series of testimonies by a gr..."
210387,1,I already knew this movie would make me cry li...
490440,1,"In my eyes, the quintessential Coen brothers f..."
315552,1,Yet another moronic Hollywood film full of con...
51495,0,Lawrence Of Arabia is a remarkable film and pa...
...,...,...
285861,0,I should tell the readers of this review that ...
463774,0,For those more familiar with the deerstalker h...
174696,1,"""Seven Psychopaths"" has a better ring to it, b..."
197438,0,Riddick is a movie about Riddick and not reall...


In [15]:
train['label'].value_counts()

label
0    18426
1     6574
Name: count, dtype: int64

In [16]:
train

,label,review_text
153501,0,If you'd like to enjoy watching Hangover 2 you...
424596,1,I remember first seeing this movie at a friend...
435503,1,Whenever it comes to a movie with a dark subje...
420288,0,I went into this movie with semi-low expectati...
286255,0,"If you don't know much about music, you might ..."
...,...,...
64638,0,Extraordinarily compelling masterpiece is the ...
219803,1,It is sort of a weird feeling when you've watc...
387421,0,I have been a fan for years and felt this movi...
135192,0,As gut-wrenching as it is emotionally upliftin...


### Change the  Dataset

In [17]:
Train = Dataset.from_pandas(train)
Eval = Dataset.from_pandas(train_eval)
Test = Dataset.from_pandas(test)

In [18]:
Train=Train.remove_columns("__index_level_0__")
Eval=Eval.remove_columns("__index_level_0__")
Test=Test.remove_columns("__index_level_0__")

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


c:\Users\Francesco\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Tokenization

In [20]:
def encodeBig(text):
    return tokenizer(text['review_text'], padding="max_length", truncation=True, max_length=128)

In [21]:
Train=Train.map(encodeBig,batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [22]:
Eval=Eval.map(encodeBig,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [23]:
Test=Test.map(encodeBig,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

### Apply the model

In [24]:
from transformers import TrainingArguments, Trainer

#### Parameters of the models

In [25]:
BATCH_SIZE = 16
WEIGHT_DECAY=0.01
LR = 2e-5
EPOCHS = 3


In [26]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)

c:\Users\Francesco\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
output_dir="D:/model/test_dir",
learning_rate=LR,
weight_decay=WEIGHT_DECAY,
num_train_epochs=EPOCHS,
evaluation_strategy="epoch",
per_device_train_batch_size=BATCH_SIZE,
per_device_eval_batch_size=BATCH_SIZE


)


In [28]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [30]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Train,
    eval_dataset=Eval,
    compute_metrics=compute_metrics,
)

In [32]:
history=trainer.train()

  0%|          | 0/4689 [00:00<?, ?it/s]

{'loss': 0.5551, 'grad_norm': 3.1969306468963623, 'learning_rate': 1.7867349114949884e-05, 'epoch': 0.32}
{'loss': 0.5295, 'grad_norm': 5.458887577056885, 'learning_rate': 1.5734698229899766e-05, 'epoch': 0.64}
{'loss': 0.5344, 'grad_norm': 2.265045404434204, 'learning_rate': 1.3602047344849649e-05, 'epoch': 0.96}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.5115514993667603, 'eval_accuracy': 0.76316, 'eval_runtime': 495.261, 'eval_samples_per_second': 50.478, 'eval_steps_per_second': 3.156, 'epoch': 1.0}
{'loss': 0.4931, 'grad_norm': 3.8840036392211914, 'learning_rate': 1.1469396459799531e-05, 'epoch': 1.28}
{'loss': 0.4682, 'grad_norm': 5.037413597106934, 'learning_rate': 9.336745574749414e-06, 'epoch': 1.6}
{'loss': 0.4734, 'grad_norm': 5.350114822387695, 'learning_rate': 7.204094689699297e-06, 'epoch': 1.92}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.5168681144714355, 'eval_accuracy': 0.75708, 'eval_runtime': 498.094, 'eval_samples_per_second': 50.191, 'eval_steps_per_second': 3.138, 'epoch': 2.0}
{'loss': 0.3965, 'grad_norm': 12.444040298461914, 'learning_rate': 5.07144380464918e-06, 'epoch': 2.24}
{'loss': 0.3525, 'grad_norm': 2.208289384841919, 'learning_rate': 2.9387929195990615e-06, 'epoch': 2.56}
{'loss': 0.344, 'grad_norm': 8.064736366271973, 'learning_rate': 8.061420345489445e-07, 'epoch': 2.88}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.6381476521492004, 'eval_accuracy': 0.72812, 'eval_runtime': 496.7928, 'eval_samples_per_second': 50.323, 'eval_steps_per_second': 3.146, 'epoch': 3.0}
{'train_runtime': 6128.0331, 'train_samples_per_second': 12.239, 'train_steps_per_second': 0.765, 'train_loss': 0.45643542829315453, 'epoch': 3.0}


In [41]:
history.metrics

{'train_runtime': 6128.0331,
 'train_samples_per_second': 12.239,
 'train_steps_per_second': 0.765,
 'total_flos': 4933332288000000.0,
 'train_loss': 0.45643542829315453,
 'epoch': 3.0}

In [33]:
evaluationRes=trainer.evaluate()

  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.6381476521492004,
 'eval_accuracy': 0.72812,
 'eval_runtime': 435.017,
 'eval_samples_per_second': 57.469,
 'eval_steps_per_second': 3.593,
 'epoch': 3.0}

In [40]:
trainer.evaluate(Test)

  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.6357943415641785,
 'eval_accuracy': 0.7298,
 'eval_runtime': 449.6723,
 'eval_samples_per_second': 55.596,
 'eval_steps_per_second': 3.476,
 'epoch': 3.0}